# Importações

In [52]:
import pandas as pd
import numpy as np

# Configura o pandas para mostrar todas as linhas (o que ativa o scroll vertical)
pd.set_option('display.max_rows', None)

# Configura o pandas para mostrar todas as colunas (o que ativa o scroll horizontal)
pd.set_option('display.max_columns', None)

# Opcional: Aumenta a largura da exibição para evitar que colunas sejam quebradas em várias linhas
pd.set_option('display.width', 1000)

# Opcional: Garante que o conteúdo de uma coluna não será truncado com "..."
pd.set_option('display.max_colwidth', None)

# 1. Métricas Monitorar com ``iqar`` e presentes no DataSUS

## 1. Tratamento de Dados

#### Lê os ``.csv``

In [ ]:
def get_monitors(cols=None):
    jan_mar_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_jan_mar.csv", encoding="latin", sep=";", usecols=cols)
    abr_jun_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_abr_jun.csv", encoding="latin", sep=";", usecols=cols)
    jul_nov_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_jul_nov.csv", encoding="latin", sep=";", usecols=cols)

    # Junta as três tabelas de monitores
    monitors_df = pd.concat([jan_mar_df, abr_jun_df, jul_nov_df], axis=0)

    return monitors_df

monitors_cols=[
    'Nome do Município', 'Estado', 'Nome da Estação', 'Sigla', 'Concentracao', 'iqar', 'Data'
]
monitors_df = get_monitors(monitors_cols)

monitors_df.rename(columns = {
    "Nome do Município": "Nome_Municipio",
    "Nome da Estação": "Nome_Estacao"
}, inplace=True)

### Filtrar os monitores com ``iqar`` preenchido

In [51]:

monitors_df['iqar'] = pd.to_numeric(monitors_df['iqar'], errors='coerce')
monitors_df = monitors_df.dropna(subset=['iqar'])
monitors_df = monitors_df[monitors_df['iqar'] >= 0]

### Classificar o ``iqar``

In [ ]:
def classify_iqar(iqar):
    if (iqar >= 0 and iqar <= 40):
        return "Boa"
    elif (iqar > 40 and iqar <= 80):
        return "Moderada"
    elif (iqar > 80 and iqar <= 120):
        return "Ruim"
    elif (iqar > 120 and iqar <= 200):
        return "Muito Ruim"
    elif (iqar > 200):
        return "Péssima"
    else:
        return np.nan
    
monitors_df['Qualidade_Ar'] = monitors_df['iqar'].apply(classify_iqar)
monitors_df.head(10)